In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from svd_compression import calculate_singular_values, compress_matrix

In [ ]:
def random_matrix(n, p=0):
    r_matrix = np.random.uniform(0, 1, (n, n))
    for ind in np.random.permutation(range(n*n))[:int(n*n*p)]:
        r_matrix[ind // n, ind % n] = 0
    return r_matrix

In [ ]:
def plot_singular_values(values):
    plt.title('Singular values')
    plt.plot(values)

In [ ]:
eps = 1e-12

In [ ]:
mat = random_matrix(256, 0.99)
v = calculate_singular_values(mat)
plot_singular_values(v)
print(mat)

In [ ]:
tree = compress_matrix(mat, v[128], 4)
mat2 = tree.eval()
print(np.max(mat2 - mat))

In [ ]:
plt.imshow(Image.fromarray(tree.draw()))

# Testy

In [ ]:
import time

In [ ]:
size = 2 ** 10
zeros_percentage = [0.99, 0.98, 0.95, 0.9, 0.8]

In [ ]:
results = []

In [ ]:
singular_indices = [1, 512, 1024]
for zeros_p in zeros_percentage:
    print('Zeros percentage:', zeros_p * 100, '%')
    matrix = random_matrix(size, zeros_p)
    
    singular_values = calculate_singular_values(matrix)
    
    plot_singular_values(singular_values)
    plt.savefig(f"saved_figures/singular_values_sparse{zeros_p}.png")
    plt.show()
    
    b_values = [1, 4]
    delta_values = [singular_values[1], singular_values[-1], singular_values[len(singular_values) // 2]]
    
    for b in b_values:
        for i, delta in enumerate(delta_values):
            time_start = time.time()
            root = compress_matrix(matrix, delta, b)
            time_stop = time.time()
            
            image = root.draw()
            plt.imshow(image)
            plt.title(f'H-matrix for b:{b}, sigma:{singular_indices[i]}')
            # plt.gray()
            plt.axis('off')
            plt.savefig(f"saved_figures/rank_{b}_delta_{i}_sparse_{zeros_p}.png")
            plt.show()
            
            decompressed_matrix = root.eval()
            norm = np.linalg.norm(matrix - decompressed_matrix) ** 2
            
            results.append((zeros_p, b, delta, time_stop - time_start, norm))

In [ ]:
import pandas as pd

dataframe = pd.DataFrame(results, columns=['Zeros number', 'Max rank', 'Delta', 'Compress time', 'Norm of difference'])

print(dataframe)

In [ ]:
def print_matrices(sparse_level):
    fig, ax = plt.subplots(3, 2, figsize=(11, 15))
    fig.tight_layout()
    for d in [0, 1, 2]:
        for ind, rank in enumerate([1, 4]):
            im = Image.open(f"saved_figures/rank_{rank}_delta_{d}_sparse_{sparse_level}.png")
            ax[d, ind].imshow(im)
            ax[d, ind].axis('off')

    plt.subplots_adjust(left=0.0,
                    bottom=0.0,
                    right=1,
                    top=1,
                    wspace=-0.25,
                    hspace=-0.25)
    plt.savefig(f"saved_figures/matrices_{sparse_level}.png")
    plt.show()



In [ ]:
print_matrices(0.8)
print_matrices(0.9)
print_matrices(0.95)
print_matrices(0.98)
print_matrices(0.99)